### קוד מבוא

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
import contextily as cx
import folium
import fiona
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point
import adjustText as aT

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

### פונקציות

In [ ]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [ ]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [ ]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [ ]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [ ]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [ ]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df

In [ ]:
def drop_geo(geoDF):
    geoDF = geoDF.drop(columns='geometry')
    return geoDF

In [ ]:
#העלת משתנים להרצת הקוד
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

### ביצוע

#### עיבוד מידע טבלאי של הדמוגרף

In [ ]:
demo=up_load_df(r'{}\background_files'.format(software_data_folder_location),'‏‏PalestiniansResults_forecast_zone')

In [ ]:
col=['male_2020',
 'female_2020',
 'male_2025',
 'female_2025',
 'male_2030',
 'female_2030',
 'male_2035',
 'female_2035',
 'male_2040',
 'female_2040',
 'male_2045',
 'female_2045',
 'male_2050',
 'female_2050']

In [ ]:
demo=demo.pivot_table(index='Proj_Area',aggfunc=sum)[col]

In [ ]:
year=['2020','2025','2030','2035','2040','2045','2050']

In [ ]:
for x in year:
    demo['pop_{}'.format(x)]= demo['female_{}'.format(x)]+demo['male_{}'.format(x)]

#### שכבת אזורי תחזית של דמוגרף

In [ ]:
proj_zones=up_load_shp(r'{}\background_files\proj_zones_pls.shp'.format(software_data_folder_location))

In [ ]:
proj_zones['Proj_Area']=proj_zones['projection']

In [ ]:
proj_zones=proj_zones[['Proj_Area', 'geometry']]

#### חיבור נתוני דמוגרף

In [ ]:
pd.merge(proj_zones,demo,on='Proj_Area',how='right').query('geometry.isna()')

 אנחנו מוכנים לוותר עליהם 
 זה ניקוז של כל הבדואיים שאין להם אזור תחזית

In [ ]:
proj_zones=pd.merge(proj_zones,demo,on='Proj_Area',how='left')

#### שכבת אזורי תנועה

In [ ]:
taz=up_load_shp(r'{}\background_files\TAZ_V4_230518_Published.shp'.format(software_data_folder_location))

In [ ]:
taz=taz.query('main_secto=="Palestinian"')[['Taz_num','geometry']]

#### שכבת בינוי

In [ ]:
bld=up_load_shp(r'{}\background_files\palestinian_bld_area_pcbs_220710.shp'.format(software_data_folder_location))

In [ ]:
bld=bld.to_crs(2039)

In [ ]:
bld=bld.dissolve()

In [ ]:
bld=bld[['geometry']]

#### הצלבת בין בינוי לבין אזורי תחזית ותנועה

In [ ]:
col=['Proj_Area',
 'geometry',
 'pop_2020',
 'pop_2025',
 'pop_2030',
 'pop_2035',
 'pop_2040',
 'pop_2045',
 'pop_2050']

In [ ]:
demo_taz_bld=bld.overlay(taz).overlay(proj_zones[col])

In [ ]:
demo_taz_bld['small_area']=demo_taz_bld.area

In [ ]:
demo_taz_bld=demo_taz_bld.set_index('Proj_Area')

In [ ]:
demo_taz_bld['proj_sum_area']=drop_geo(demo_taz_bld).groupby(by='Proj_Area').sum()[['small_area']]

In [ ]:
demo_taz_bld['pre_from_proj']=demo_taz_bld['small_area']/demo_taz_bld['proj_sum_area']

In [ ]:
pop_year=['pop_2020',
 'pop_2025',
 'pop_2030',
 'pop_2035',
 'pop_2040',
 'pop_2045',
 'pop_2050']

In [ ]:
for y in pop_year:
    demo_taz_bld['{}'.format(y)]=demo_taz_bld['{}'.format(y)]*demo_taz_bld['pre_from_proj']

בדיקה שלא הומצא כמות אנשים

In [ ]:
demo_taz_bld[pop_year].sum().sum()-proj_zones[pop_year].sum().sum()

#### סכום אנשים לפי אזורי תנועה

In [ ]:
taz_demo_pls=drop_geo(demo_taz_bld).pivot_table(index='Taz_num',aggfunc=sum)

In [ ]:
add_year=['2025', '2030', '2035', '2040', '2045', '2050']

In [ ]:
x=2020
for y in add_year:
    taz_demo_pls['precent_add_pop_{}'.format(y)]=taz_demo_pls['pop_{}'.format(y)]/taz_demo_pls['pop_{}'.format(str(x))]
    taz_demo_pls['precent_add_pop_{}'.format(y)]=taz_demo_pls['precent_add_pop_{}'.format(y)].round(2)
    taz_demo_pls['precent_add_pop_{}'.format(y)].fillna(0)
    x+=5

#### ייצוא

In [ ]:
col=['pop_2020',
 'precent_add_pop_2025',
 'precent_add_pop_2030',
 'precent_add_pop_2035',
 'precent_add_pop_2040',
 'precent_add_pop_2045',
 'precent_add_pop_2050']

In [ ]:
r'{}\Intermediates\taz_demo_pls_2020_and_pre_growth_till_2050.xlsx'.format(software_data_folder_location)

In [ ]:
taz_demo_pls[col].to_excel(r'{}\Intermediates\taz_demo_pls_2020_and_pre_growth_till_2050.xlsx'.format(software_data_folder_location))